Highest score: 0.656700	

In [78]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score

def read_data_from_csv(path):
    """Load datasets from CSV files.
    Args:
        path (str): Path to the CSV file.
    Returns:
        X (np.ndarray): Features of samples.
        y (np.ndarray): Labels of samples, only provided in the public datasets.
    """
    assert os.path.exists(path), f'File not found: {path}!'
    assert os.path.splitext(path)[
        -1] == '.csv', f'Unsupported file type {os.path.splitext(path)[-1]}!'

    data = pd.read_csv(path)
    column_list = data.columns.values.tolist()

    if 'Label' in column_list:
        # for the public dataset, label column is provided.
        column_list.remove('Label')
        X = data[column_list].values
        y = data['Label'].astype('int').values
        return X, y
    else:
        # for the private dataset, label column is not provided.
        X = data[column_list].values
        return X
    

X_public, y_public = read_data_from_csv('assignment_5_public.csv')
print('Shape of X_public:', X_public.shape)  # n_sample, m_feature (30000, 58)
print('Shape of y_public:', y_public.shape)  # n_sample (30000,)

'''
CODE HERE!
'''






Shape of X_public: (30000, 58)
Shape of y_public: (30000,)


'\nCODE HERE!\n'

|   Features                    |    Types	        |       data types      | 
|-------------------------------|-------------------|-----------------------|
|       1-11, 18-29, 38-58      |    Continuous     |       numberic number |
|       12-17, 30-37            |   Categorical     |       boolean(1/0)    |

In [70]:
df_x = pd.DataFrame(X_public)
df_y = pd.DataFrame(y_public)

In [72]:
df_y

,0
0,0
1,0
2,1
3,1
4,0
...,...
29995,0
29996,1
29997,0
29998,1


In [77]:
print(df_y[0].value_counts())

0
1    15978
0    14022
Name: count, dtype: int64


In [12]:
# try construct the model without any preprocess

k_folds = KFold(n_splits = 8)

logisticReg_model = LogisticRegression(solver = 'liblinear', class_weight = 'balanced', max_iter =300, penalty = 'l1')

scores = cross_val_score(logisticReg_model, X_public, y_public, cv = k_folds)

print("Cross Validation Scores: ", scores)
print("\nAverage CV Score: ", scores.mean())

Cross Validation Scores:  [0.65786667 0.65066667 0.66426667 0.64906667 0.64453333 0.64506667
 0.66506667 0.6496    ]

Average CV Score:  0.6532666666666667


In [64]:
normalized_x = df_x
# Continuous features   1-11, 18-29, 38-58
df_1T11 = df_x.iloc[:, 0:11]  
df_18T19 = df_x.iloc[:, 17:29]
df_38T58 = df_x.iloc[:, 37:58]

# Categorical features  12-17, 30-37 
df_12T17 = df_x.iloc[:, 11:17]  
df_30T37 = df_x.iloc[:, 29:37]  

In [65]:
# concatenating along columns
horizontal_Continuous_concat = pd.concat([df_1T11, df_18T19, df_38T58], axis=1)
horizontal_Categorical_concat = pd.concat([df_12T17, df_30T37], axis=1)

# Normalize the numeric features
scaler = MinMaxScaler()
normalized_Continuous = pd.DataFrame(scaler.fit_transform(pd.DataFrame(horizontal_Continuous_concat)))

# change Categorical features data type from float to boolean
for column in horizontal_Categorical_concat:
    horizontal_Categorical_concat[column] = horizontal_Categorical_concat[column].astype(bool)

print(horizontal_Continuous_concat.shape)
print(horizontal_Categorical_concat.shape)

(30000, 44)
(30000, 14)


In [66]:
#  combine to full dataset
df_fullConcat = pd.concat([normalized_Continuous, horizontal_Categorical_concat], axis=1)

In [67]:
df_fullConcat

,0,1,2,3,4,5,6,7,8,9,...,15,16,29,30,31,32,33,34,35,36
0,0.285714,0.022233,0.000745,7.309030e-11,0.000887,0.049342,0.017241,0.085938,0.010989,0.303731,...,False,False,False,True,False,False,False,False,False,False
1,0.285714,0.022706,0.000787,7.217675e-11,0.001143,0.019737,0.034483,0.000000,0.000000,0.271099,...,False,False,False,False,False,True,False,False,False,False
2,0.333333,0.029092,0.000644,7.426417e-11,0.000835,0.029605,0.034483,0.007812,0.000000,0.309443,...,True,False,False,False,False,False,False,False,True,True
3,0.285714,0.015137,0.000901,6.874928e-11,0.001254,0.013158,0.034483,0.007812,0.021978,0.201404,...,False,False,True,False,False,False,False,False,False,False
4,0.571429,0.147706,0.000405,7.876849e-11,0.000689,0.128289,0.060345,0.085938,0.000000,0.309256,...,False,True,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,0.523810,0.051325,0.000631,7.620845e-11,0.000969,0.013158,0.017241,0.007812,0.010989,0.284189,...,False,False,True,False,False,False,False,False,False,False
29996,0.571429,0.070009,0.000417,7.746878e-11,0.000633,0.029605,0.051724,0.023438,0.000000,0.258694,...,True,False,False,False,False,True,False,False,False,False
29997,0.333333,0.036306,0.000677,7.514986e-11,0.000991,0.019737,0.025862,0.007812,0.010989,0.301023,...,False,False,False,False,False,False,True,False,False,False
29998,0.428571,0.178571,0.000443,7.901729e-11,0.000733,0.095395,0.129310,0.054688,0.000000,0.342777,...,True,False,True,False,False,False,False,False,False,False


In [68]:
# try construct the model again

k_folds = KFold(n_splits = 8)

logisticReg_model = LogisticRegression(solver = 'liblinear', class_weight = 'balanced', max_iter =300, penalty = 'l1')

scores = cross_val_score(logisticReg_model, df_fullConcat, y_public, cv = k_folds)

print("Cross Validation Scores: ", scores)
print("\nAverage CV Score: ", scores.mean())

Cross Validation Scores:  [0.65653333 0.6568     0.66026667 0.64666667 0.64266667 0.6488
 0.66213333 0.6512    ]

Average CV Score:  0.6531333333333333


In [69]:
# try construct the model again

k_folds = KFold(n_splits = 8)
# use L2 as penalty
logisticReg_model = LogisticRegression(solver = 'liblinear', class_weight = 'balanced', max_iter =300, penalty = 'l2')

scores = cross_val_score(logisticReg_model, df_fullConcat, y_public, cv = k_folds)

print("Cross Validation Scores: ", scores)
print("\nAverage CV Score: ", scores.mean())

Cross Validation Scores:  [0.65306667 0.64826667 0.65466667 0.64266667 0.63013333 0.6464
 0.65786667 0.64586667]

Average CV Score:  0.6473666666666666


In [ ]:
X_private = read_data_from_csv('assignment_5_private.csv')
print('Shape of X_private:', X_private.shape)  # k_sample, m_feature (5000, 58)



# remove and make your own predictions.
preds = np.full(len(X_private), -1,
                dtype=int)
'''
CODE HERE!
e.g.,
preds = np.full(len(X_private), -1, dtype=int)
'''

In [ ]:
submission = pd.DataFrame({'Label': preds})
submission.to_csv('assignment_5.csv', index=True, index_label='Id')

In [80]:
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet'],
    'C': [0.00001, 0.001, 0.1, 1, 5, 10, 100, 300],
    'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'], 
    'max_iter': [100, 200, 300]
    
}

grid_search = GridSearchCV(estimator=LogisticRegression(), param_grid=param_grid, cv=8)


grid_search.fit(df_fullConcat, df_y)

# Get the best parameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

c:\Users\lucas\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\lucas\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\lucas\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\lucas\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1184: DataC

In [81]:
fullParamGrid_result = pd.DataFrame(grid_search.cv_results_)
df_paramGrid = fullParamGrid_result[['param_C', 'param_max_iter', 'param_penalty',
    'param_solver', 'mean_test_score', 'rank_test_score']]
# Drop rows that has NaN values on selected columns
df_paramGrid = df_paramGrid.dropna(subset=['mean_test_score'])
df_paramGrid.sort_values(by='mean_test_score', ascending=False,).head(20)

,param_C,param_max_iter,param_penalty,param_solver,mean_test_score,rank_test_score
401,300,200,l1,saga,0.656700,1
289,10,200,l1,liblinear,0.656667,2
307,10,300,l1,liblinear,0.656667,2
347,100,200,l1,saga,0.656667,4
271,10,100,l1,liblinear,0.656633,5
217,5,100,l1,liblinear,0.656600,6
407,300,200,l2,saga,0.656600,6
311,10,300,l1,saga,0.656600,6
424,300,300,l2,sag,0.656567,9
406,300,200,l2,sag,0.656567,9


|   param_solver       |    param_penalty	|   param_C             |   |   |
|----------------------|--------------------|-----------------------|---|---|
|       saga	       |        l1          |   300	    	        |   |   |
|      liblinear       |        l2          |   10	    	        |   |   |


In [82]:
fullParamGrid_result.to_csv('logModel_gridResult.csv', index=True)

In [98]:
df_public =  pd.read_csv('assignment_5_public.csv')
df_corr = df_public
# Specify the threshold correlation value for dropping columns
threshold = 0.07

# Calculate the correlation between each column and the target label
correlations = df_corr.drop('Label', axis=1).apply(lambda x: x.corr(df_corr['Label']))

correlations

Feature 1    -0.049610
Feature 2     0.046903
Feature 3     0.004154
Feature 4     0.005409
Feature 5     0.003711
Feature 6     0.095556
Feature 7     0.044428
Feature 8     0.069973
Feature 9     0.006962
Feature 10   -0.035898
Feature 11    0.078741
Feature 12    0.031088
Feature 13   -0.117457
Feature 14    0.000820
Feature 15    0.115134
Feature 16    0.103499
Feature 17   -0.152720
Feature 18    0.049052
Feature 19    0.023133
Feature 20    0.033002
Feature 21    0.003614
Feature 22   -0.033782
Feature 23    0.009164
Feature 24    0.089692
Feature 25    0.065805
Feature 26    0.154622
Feature 27    0.057683
Feature 28    0.061813
Feature 29    0.067775
Feature 30   -0.022099
Feature 31   -0.043569
Feature 32   -0.043699
Feature 33   -0.028032
Feature 34    0.013335
Feature 35    0.108654
Feature 36    0.087328
Feature 37    0.143174
Feature 38    0.071899
Feature 39   -0.084083
Feature 40   -0.159371
Feature 41    0.059058
Feature 42    0.092818
Feature 43    0.108985
Feature 44 

In [99]:
# Identify the columns with correlations below the threshold
low_corr_columns = correlations[abs(correlations) < threshold].index.tolist()

# Drop the columns with low correlation from the DataFrame
df_corr = df_corr.drop(low_corr_columns, axis=1)

# Print the updated DataFrame
df_corr

,Feature 6,Feature 11,Feature 13,Feature 15,Feature 16,Feature 17,Feature 24,Feature 26,Feature 35,Feature 36,...,Feature 38,Feature 39,Feature 40,Feature 42,Feature 43,Feature 44,Feature 45,Feature 47,Feature 48,Label
0,15.0,7.0,0.0,0.0,0.0,0.0,2560.823529,4641.205086,0.0,0.0,...,0.313978,0.183855,0.028572,0.028572,0.604654,0.018615,0.024272,0.625000,0.375000,0
1,6.0,8.0,0.0,0.0,0.0,0.0,0.000000,1813.918964,0.0,0.0,...,0.583456,0.025096,0.025019,0.341370,0.274729,-0.049188,0.014286,0.500000,0.500000,0
2,9.0,9.0,0.0,0.0,1.0,0.0,749.000000,2500.480540,0.0,1.0,...,0.022274,0.022223,0.022247,0.911034,0.301486,0.033362,0.026515,0.388889,0.611111,1
3,4.0,9.0,1.0,0.0,0.0,0.0,3285.442902,5615.491585,0.0,0.0,...,0.022247,0.022796,0.022223,0.135477,0.425568,0.190057,0.034247,0.833333,0.166667,1
4,39.0,10.0,0.0,0.0,0.0,1.0,0.000000,2852.910759,0.0,0.0,...,0.020006,0.020077,0.516807,0.322620,0.382016,0.178356,0.051302,0.773810,0.226190,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,4.0,8.0,1.0,0.0,0.0,0.0,0.000000,2899.935449,0.0,0.0,...,0.025001,0.899946,0.025000,0.025000,0.460830,0.051290,0.050885,0.638889,0.361111,0
29996,9.0,9.0,0.0,0.0,1.0,0.0,1956.909091,2720.447144,0.0,0.0,...,0.022233,0.192556,0.022588,0.577674,0.511503,0.150920,0.052459,0.842105,0.157895,1
29997,6.0,4.0,0.0,0.0,0.0,0.0,2461.102849,3467.658153,0.0,0.0,...,0.799999,0.050000,0.050001,0.050000,0.484780,0.136700,0.043077,0.700000,0.300000,0
29998,29.0,10.0,0.0,0.0,1.0,0.0,2600.000000,3398.157826,0.0,0.0,...,0.020105,0.120723,0.121225,0.717921,0.489340,0.125692,0.048429,0.831461,0.168539,1


In [101]:
k_folds = KFold(n_splits = 8)
# use L2 as penalty
logisticReg_model = LogisticRegression(solver = 'saga', class_weight = 'balanced', max_iter =300, penalty = 'l1')


x_train, y_train = df_corr.drop('Label', axis=1), df_public['Label']
scores = cross_val_score(logisticReg_model, x_train, y_train, cv = k_folds)

print("Cross Validation Scores: ", scores)
print("\nAverage CV Score: ", scores.mean())

c:\Users\lucas\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\lucas\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\lucas\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\lucas\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\lucas\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the 

Cross Validation Scores:  [0.53626667 0.52186667 0.5472     0.53253333 0.52053333 0.52746667
 0.53973333 0.53706667]

Average CV Score:  0.5328333333333333


c:\Users\lucas\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [103]:
logisticReg_model = LogisticRegression(solver = 'liblinear', class_weight = 'balanced', max_iter =300, penalty = 'l1')


x_train, y_train = df_corr.drop('Label', axis=1), df_corr['Label']
scores = cross_val_score(logisticReg_model, x_train, y_train, cv = k_folds)

print("Cross Validation Scores: ", scores)
print("\nAverage CV Score: ", scores.mean())

Cross Validation Scores:  [0.65413333 0.6408     0.65093333 0.63093333 0.6336     0.62906667
 0.6552     0.63973333]

Average CV Score:  0.6417999999999999
